# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [2]:
import fullcontrol as fc
import math

In [3]:
def circle_infill(diameter:float,start_point:fc.Point,line_space:float,direction:float):
    import math
    import numpy as np
    radius_c=diameter/2
    steps=[]
    flag=True
    steps.append(start_point)
    steps.append(fc.polar_to_point(centre=steps[-1],radius=line_space,angle=direction))
    for space in np.arange(line_space,diameter,line_space):
        if flag:
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=line_space,angle=direction))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
        else:
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=line_space,angle=direction))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))            
        flag=not flag       
    return steps

In [4]:
def circle_infill_arc(diameter:float,start_point:fc.Point,centre_point:fc.Point,line_space:float,direction:float):
    import math
    import numpy as np
    radius_c=diameter/2
    steps=[]
    flag=True
    steps.append(start_point)
    steps.append(fc.polar_to_point(centre=steps[-1],radius=line_space,angle=direction))
    for space in np.arange(line_space,diameter,line_space):
        if flag:
                arc_endX=fc.polar_to_point(centre=steps[-1],radius=line_space,angle=direction)
                arc_end=fc.polar_to_point(centre=arc_endX, radius=math.sqrt(math.fabs((radius_c**2)-(radius_c-2*space)**2)),angle=direction+math.radians(90))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))
                arc_start=fc.point_to_polar(target_point=steps[-1],origin_point=centre_point)
                arc_move=fc.angleXY_between_3_points(steps[-1],centre_point,arc_end)            
                steps.extend(fc.arcXY(centre=centre_point, radius=radius_c,start_angle=arc_start.angle,arc_angle=arc_move))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
                


                
        else:
                arc_endX=fc.polar_to_point(centre=steps[-1],radius=line_space,angle=direction)
                arc_end=fc.polar_to_point(centre=arc_endX, radius=math.sqrt(math.fabs((radius_c**2)-(radius_c-2*space)**2)),angle=direction-math.radians(90))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction-math.radians(90)))
                arc_start=fc.point_to_polar(target_point=steps[-1],origin_point=centre_point)
                arc_move=fc.angleXY_between_3_points(steps[-1],centre_point,arc_end)            
                steps.extend(fc.arcXY(centre=centre_point, radius=radius_c,start_angle=arc_start.angle,arc_angle=arc_move))
                steps.append(fc.polar_to_point(centre=steps[-1], radius=math.sqrt((radius_c**2)-(radius_c-space)**2),angle=direction+math.radians(90)))
                           
        flag=not flag       
    return steps

In [5]:
# printer/gcode parameters

design_name = 'cylinder'
nozzle_temp = 0
bed_temp = 0
print_speed = 300
fan_percent = 0
printer_name='stoneflower3d' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [14]:
# design parameters

EW = 0.5 # extrusion width
EH = 0.25 # extrusion height (and layer height)
initial_z = EH*1 # initial nozzle position is set to 1x the extrusion height to get a bit of 'squish' for good bed adhesion
height=0.5
layers = int(height//EH)
dia=20 #diameter of the cylinder


In [ ]:
# generate the design (make sure you've run the above cells before running this cell)
initial_xyz=fc.Point(x=200, y=200, z=initial_z)
steps = []
#Generate the layer patterns such that no travel is require between the layers
for layer in range(layers):
        
        
        layer1=[]
        centre_point = fc.move(initial_xyz,fc.Vector(x=dia/2,y=dia/2,z=EH*layer))
        radius = dia/2
        start_angle =0
        segments = 32
        clockwise = True
        circle_steps = fc.circleXY(centre_point, radius, start_angle, segments, clockwise)        
        first_point=fc.move(initial_xyz,fc.Vector(y=radius,z=EH*layer))
        line_spacing = 1        
        infill_steps = circle_infill(dia-2*EW,first_point,line_spacing,0)              
        layer1.extend(infill_steps)
        layer1.extend(circle_steps)
        print(layer1[-1])
           
    
        layer2=[]
        centre_point = fc.move(initial_xyz,fc.Vector(x=dia/2,y=dia/2,z=EH*layer))
        radius = dia/2
        start_angle =0
        segments = 32
        clockwise = False        
        arc_steps=fc.arcXY(centre_point,radius,start_angle,-1.5*math.pi,segments)
        line_spacing = 1 
        first_point=fc.move(initial_xyz,fc.Vector(x=radius,y=2*radius,z=EH*layer))
        infill_steps = circle_infill(dia-2*EW,first_point,line_spacing,1.5*math.pi)       
        layer2.extend(arc_steps)
        layer2.extend(infill_steps)
        
        
         
        
        layer3=[]
        centre_point = fc.move(initial_xyz,fc.Vector(x=dia/2,y=dia/2,z=EH*layer))
        radius = dia/2
        start_angle =1.5*math.pi
        segments = 32
        clockwise = True      
        first_point=fc.move(initial_xyz,fc.Vector(y=radius,z=EH*layer))
        line_spacing = 1        
        infill_steps = circle_infill(dia-2*EW,first_point,line_spacing,0)                    
        arc_steps=(fc.arcXY(centre_point,radius,start_angle,1.5*math.pi,segments))       
        layer3.extend(arc_steps)
        layer3.extend(infill_steps)
        

        # layer4=[]
        # centre_point = fc.move(initial_xyz,fc.Vector(x=dia/2,y=dia/2,z=EH*layer))
        # radius = dia/2
        # start_angle =math.pi
        # segments = 32
        # clockwise = True
        # circle_steps = fc.circleXY(centre_point, radius, start_angle, segments, clockwise)        
        # first_point=circle_steps[-1]
        # line_spacing = 1        
        # infill_steps = circle_infill(dia-2*EW,first_point,line_spacing,0)
        # layer4.extend(circle_steps)
        # layer4.extend(infill_steps)
        
        layertypes=[layer1,layer2,layer3]
        steps.extend(layertypes[layer%len(layertypes)]) 

#steps = fc.move(rectangle_steps, fc.Vector(z=EH), copy=True, copy_quantity=layers)

x=220.0 y=210.0 z=0.5 color=None
x=220.0 y=210.0 z=0.75 color=None


In [18]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [ ]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol